## Code for automatic merging of Mozambique data

Some code to process XLS data and merge it into an appropriate format.

**After longer work on this - retreat! - do manual merging**

In [ ]:
import os
import xlrd
import numpy as np
import pandas as pd
from termcolor import colored
import unidecode
import difflib

In [ ]:
file_path = "C://Users//wb555300//OneDrive - WBG//Municipality_merge//Source_tables"
os.chdir(file_path)
os.getcwd()

#### Functions:

In [ ]:
#Create a function to check if format is xls:
def is_xls(files):
    xls_format = []
    for x in files:
        leng = len(x.strip().split("."))
        xls_format.append(x.strip().split(".")[leng-1]=="xls")
    return np.array(xls_format)


# split at multiple different seperators:
def multi_split(txt, seps):
    default_sep = seps[0]

    # we skip seps[0] because that's the default separator
    for sep in seps[1:]:
        txt = txt.replace(sep, default_sep)
    return [i.strip() for i in txt.split(default_sep)]


## Functions to identify non-empty rows and columns:

def nonempty_rows(sheet):
    empty_row = []
    for row in range(sheet.nrows):
        empty_row.append(all(np.array(sheet.row_values(row,0))==''))
    return np.where(np.array(empty_row)==False)[0]

def nonempty_cols(sheet):
    empty_col = []
    for col in range(sheet.ncols):
        empty_col.append(all(np.array(sheet.col_values(col,0))==''))
    return np.where(np.array(empty_col)==False)[0]

# Code to open a respective table:
def open_table(muni, table, path):
    path_loc = path + muni + '_Table_' + str(table) +'.csv'
    return pd.read_csv(path_loc, index_col=0)


#### Code:

In [ ]:
#Make list with all files in directory
all_files = np.array(os.listdir())
xls_files = all_files[is_xls(all_files)]
print(xls_files)

In [ ]:
xls_files[0:2]

### Extract all tables from one file:

Extract current municipality name:

In [ ]:
out_loc = 'C:/Users/wb555300/OneDrive - WBG/Municipality_merge/Version2/'

## Save list with problematic files
problem_files = []

## List with municipalities:
munis = []
# and available municipalities
munis_avail = []


for file in xls_files:

    #Extract muni name:
    muni_name = multi_split(file, seps = [" ", "_", "."])[0]
    munis.append(muni_name)
    print('Currently processing data on: ', muni_name)
    
    #Open file:
    wb = xlrd.open_workbook(file)
    sheet = wb.sheet_by_index(0) 

    # Extract number of rows and columns in file:
    ne_rows = nonempty_rows(sheet)
    ne_cols = nonempty_cols(sheet)

    # identify rows and columns where table starts (all tables tart with DESCRICAO):
    start_rowcol = []

    for row in ne_rows:
        for col in ne_cols:
            if(sheet.cell_value(row, col)=="DESCRICAO"): 
                start_rowcol.append([row, col])


    #Extract all tables:
    table_count = 1

    for c_start in start_rowcol:

        year_row = c_start[0]
        OE_row = c_start[0]+1

        years = sheet.row_values(year_row, c_start[1]+1)
        OEs = sheet.row_values(OE_row, c_start[1]+1)
        OEs = list(map(lambda x: unidecode.unidecode(x), OEs))

        # duplicate year vector:
        for x in range(len(years)):
            if years[x] == '':
                years[x] = years[x-1]

        i = 2
        current_rowname_start = multi_split(sheet.cell_value(c_start[0]+i, c_start[1]), [" ", "_", "."])[0]

        temp_df = pd.DataFrame({'muni': np.repeat(muni_name, len(years)),'year': np.array(years), 'OE': np.array(OEs)})
        temp_df_cols = 3

        while current_rowname_start != "TOTAL":
            current_rowname_start = multi_split(sheet.cell_value(c_start[0]+i, c_start[1]), [" ", "_", "."])[0]
            if sheet.cell_value(c_start[0]+i, c_start[1]-1)!='':
                temp_num = sheet.cell_value(c_start[0]+i, c_start[1]-1)
            temp_name = str(temp_num).replace('.','_')+'-'+unidecode.unidecode(sheet.cell_value(c_start[0]+i, c_start[1])).upper().strip().replace("  "," ").replace(" ", "_").replace(",", "_")
            temp_values = sheet.row_values(c_start[0]+i, c_start[1]+1)
                      
            temp_df.insert(temp_df_cols, temp_name, temp_values, True)

            temp_df_cols = temp_df_cols +1
            i = i+1

        csv_file_path = out_loc + muni_name + '_Table_' + str(table_count) +'.csv'
    
        temp_df.to_csv(csv_file_path)
        table_count = table_count + 1
     
    if table_count-1 != 3: 
        print(colored(str(table_count-1) + ' tables found \n', color='red'))
        problem_files.append(file)
    else:     
        print(colored(str(table_count-1) + ' tables found \n'))
        munis_avail.append(muni_name)
        
    

In [ ]:
print("Problem files: ", [multi_split(file, seps = [" ", "_", "."])[0] for file in problem_files] ,"\n")

### Simply merge tables:
In the following all tables are merged independently of similar column names. The code to replace similar column names can be found below... It was not used in the end as there were too many similarities in column names that needs to be judged on a one by one basis.

#### Table 1

In [ ]:
overview_table_1 = open_table(munis_avail[0],1, path = out_loc)

for muni in munis_avail:
    print("processing " + muni)
    temp_table = open_table(muni,1, out_loc).iloc[0:14]
    #temp_table = trans_similar_columns(overview_table_1, temp_table)
    overview_table_1 = overview_table_1.append(temp_table, sort = False)
    print("\n")

overview_table_1.to_csv("../Overview/Table_1/Overview_Table_1.csv")

#### Table 2

In [ ]:
overview_table_2 = open_table(munis_avail[0],2, path = out_loc)

for muni in munis_avail[1:]:
    print("processing " + muni)
    temp_table = open_table(muni,2, out_loc).iloc[0:14]
    #temp_table = trans_similar_columns(overview_table_1, temp_table)
    overview_table_2 = overview_table_2.append(temp_table, sort = False)
    print("\n")

overview_table_2.to_csv("../Overview/Table_2/Overview_Table_2.csv")

#### Table 3

In [ ]:
overview_table_3 = open_table(munis_avail[0],3, path = out_loc)

for muni in munis_avail[1:]:
    print("processing " + muni)
    temp_table = open_table(muni,3, out_loc).iloc[0:14]
    #temp_table = trans_similar_columns(overview_table_3, temp_table)
    overview_table_3 = overview_table_3.append(temp_table, sort = False)
    print("\n")

overview_table_3.to_csv("../Overview/Table_3/Overview_Table_3.csv")

### More automation by automatic column name replacement (NOT USED):

Function to replace similar column names - only if initial identifier agrees!

In [ ]:
def trans_similar_columns(df1, df2, threshold = 0.8, trace = True):
    ### Function compares column names of df2 with the names of df1 and replaces 
    ### names in df2 with the names of df1 if they are sufficiently similar!

    col1 = list(df1.columns)[3:]
    col2 = list(df2.columns)[3:]

    col2_new = col2.copy()

    for i2 in range(len(col2)):
        #Iterate through columns of the existing data frame:

        indicator1 = np.array([(col2_new[i2].split("-")[0] == col1[i1].split("-")[0]) for i1 in range(len(col1))])
        indicator2 = np.array([difflib.SequenceMatcher(lambda z: z == " ", (col2_new[i2].split("-"))[1],col1[i1].split("-")[1]).ratio() for i1 in range(len(col1))])        
        indicator_t = indicator1*indicator2 > threshold 

        if any(indicator_t):
            indi = int(np.where(indicator2 == indicator2.max())[0][0])
            col2_new[i2] = col1[indi]
            if indicator2.max() != 1 and trace == True:
                print(col2_new[i2], 'was replaced with', col1[indi])
        else:
            temp = np.where(indicator2 == indicator2.max())[0][0]
            indi = int(temp)
            if trace == True:
                print(colored(str(col2_new[i2])+' has highest similarity of '+ str(indicator2.max().round(2)) +' with ' + str(col1[indi]), color = "red"))

    df2.columns = list(df2.columns)[:3]+col2_new
    return df2

In [ ]:
## Merge all Data Frames

# ERRORS OCCUR DUE TO DUPLICATE COLUMN NAMES

overview_table_1 = open_table(munis_avail[0],1, path = out_loc)

for muni in munis_avail:
    print("processing " + muni)
    temp_table = open_table(muni,1, out_loc).iloc[0:14]
    temp_table = trans_similar_columns(overview_table_1, temp_table)
    overview_table_1 = overview_table_1.append(temp_table, sort = False)
    print("\n")

overview_table_1.to_csv("../Overview/Table_1/Overview_Table_1.csv")